<a href="https://colab.research.google.com/github/cmd-max/HumanFall/blob/main/Problem1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to SQLite database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Create tables
c.execute('''
CREATE TABLE DimCustomer (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT,
    ContactNumber TEXT,
    Email TEXT,
    ShippingAddress TEXT,
    RegistrationDate DATE
)
''')

c.execute('''
CREATE TABLE DimProduct (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Category TEXT
)
''')

c.execute('''
CREATE TABLE DimProductVariant (
    ProductVariantID INTEGER PRIMARY KEY,
    ProductID INTEGER,
    VariantName TEXT,
    LaunchDate DATE,
    DiscontinueDate DATE,
    FOREIGN KEY (ProductID) REFERENCES DimProduct(ProductID)
)
''')

c.execute('''
CREATE TABLE DimDate (
    DateID INTEGER PRIMARY KEY,
    Date DATE,
    Year INTEGER,
    Month INTEGER,
    Day INTEGER,
    Quarter INTEGER
)
''')

c.execute('''
CREATE TABLE DimPriceHistory (
    PriceHistoryID INTEGER PRIMARY KEY,
    ProductVariantID INTEGER,
    EffectiveDate INTEGER,
    Price REAL,
    FOREIGN KEY (ProductVariantID) REFERENCES DimProductVariant(ProductVariantID),
    FOREIGN KEY (EffectiveDate) REFERENCES DimDate(DateID)
)
''')

c.execute('''
CREATE TABLE DimOrderStatus (
    OrderStatusID INTEGER PRIMARY KEY,
    Status TEXT
)
''')

c.execute('''
CREATE TABLE FactOrder (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    ProductVariantID INTEGER,
    OrderDateID INTEGER,
    OrderStatusID INTEGER,
    Quantity INTEGER,
    TotalAmount REAL,
    FOREIGN KEY (CustomerID) REFERENCES DimCustomer(CustomerID),
    FOREIGN KEY (ProductVariantID) REFERENCES DimProductVariant(ProductVariantID),
    FOREIGN KEY (OrderDateID) REFERENCES DimDate(DateID),
    FOREIGN KEY (OrderStatusID) REFERENCES DimOrderStatus(OrderStatusID)
)
''')

# Insert data into DimDate
start_date = datetime(2022, 1, 1)
date_list = [(i, start_date + timedelta(days=i)) for i in range(730)]
date_rows = [(date[0], date[1].strftime('%Y-%m-%d'), date[1].year, date[1].month, date[1].day, (date[1].month-1)//3 + 1) for date in date_list]
c.executemany('INSERT INTO DimDate VALUES (?, ?, ?, ?, ?, ?)', date_rows)

# Insert data into DimOrderStatus
order_statuses = [(1, 'Pending'), (2, 'Shipped'), (3, 'Delivered'), (4, 'Cancelled')]
c.executemany('INSERT INTO DimOrderStatus VALUES (?, ?)', order_statuses)

# Insert data into DimCustomer
customers = [
    (i, f'Customer{i}', f'123-456-789{i}', f'customer{i}@example.com', f'Address {i}', start_date + timedelta(days=random.randint(0, 730)))
    for i in range(1, 11)
]
c.executemany('INSERT INTO DimCustomer VALUES (?, ?, ?, ?, ?, ?)', customers)

# Insert data into DimProduct and DimProductVariant
products = [
    (1, 'ProductA', 'Category1'),
    (2, 'ProductB', 'Category1'),
    (3, 'ProductC', 'Category2'),
    (4, 'ProductD', 'Category2'),
    (5, 'ProductE', 'Category3'),
    (6, 'ProductF', 'Category3'),
    (7, 'ProductG', 'Category4'),
    (8, 'ProductH', 'Category4'),
    (9, 'ProductI', 'Category5'),
    (10, 'ProductJ', 'Category5')
]
c.executemany('INSERT INTO DimProduct VALUES (?, ?, ?)', products)

product_variants = [
    (1, 1, 'VariantA1', start_date.strftime('%Y-%m-%d'), None),
    (2, 1, 'VariantA2', (start_date + timedelta(days=365)).strftime('%Y-%m-%d'), None),
    (3, 2, 'VariantB1', start_date.strftime('%Y-%m-%d'), None),
    (4, 2, 'VariantB2', (start_date + timedelta(days=180)).strftime('%Y-%m-%d'), None),
    (5, 3, 'VariantC1', start_date.strftime('%Y-%m-%d'), None),
    (6, 4, 'VariantD1', start_date.strftime('%Y-%m-%d'), None),
    (7, 5, 'VariantE1', start_date.strftime('%Y-%m-%d'), None),
    (8, 6, 'VariantF1', start_date.strftime('%Y-%m-%d'), None),
    (9, 7, 'VariantG1', start_date.strftime('%Y-%m-%d'), None),
    (10, 8, 'VariantH1', start_date.strftime('%Y-%m-%d'), None)
]
c.executemany('INSERT INTO DimProductVariant VALUES (?, ?, ?, ?, ?)', product_variants)

# Insert data into DimPriceHistory
price_history = [
    (i, i, random.randint(0, 730), random.uniform(10.0, 500.0))
    for i in range(1, 11)
]
c.executemany('INSERT INTO DimPriceHistory VALUES (?, ?, ?, ?)', price_history)

# Insert data into FactOrder
orders = [
    (
        i,
        random.randint(1, 10),  # CustomerID
        random.randint(1, 10),  # ProductVariantID
        random.randint(0, 729), # OrderDateID
        random.randint(1, 4),   # OrderStatusID
        random.randint(1, 5),   # Quantity
        random.uniform(50.0, 1500.0)  # TotalAmount
    )
    for i in range(1, 1001)
]
c.executemany('INSERT INTO FactOrder VALUES (?, ?, ?, ?, ?, ?, ?)', orders)

# Commit and close
conn.commit()
conn.close()


In [3]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Function to print all rows from a query
def print_query(query):
    c.execute(query)
    rows = c.fetchall()
    for row in rows:
        print(row)

# Query and print DimCustomer table
print("DimCustomer:")
print_query("SELECT * FROM DimCustomer")

# Query and print FactOrder table (limit to 10 rows)
print("\nFactOrder (first 10 rows):")
print_query("SELECT * FROM FactOrder LIMIT 10")

# Close the connection
conn.close()



DimCustomer:
(1, 'Customer1', '123-456-7891', 'customer1@example.com', 'Address 1', '2023-08-11 00:00:00')
(2, 'Customer2', '123-456-7892', 'customer2@example.com', 'Address 2', '2022-08-31 00:00:00')
(3, 'Customer3', '123-456-7893', 'customer3@example.com', 'Address 3', '2022-09-07 00:00:00')
(4, 'Customer4', '123-456-7894', 'customer4@example.com', 'Address 4', '2023-01-02 00:00:00')
(5, 'Customer5', '123-456-7895', 'customer5@example.com', 'Address 5', '2022-01-15 00:00:00')
(6, 'Customer6', '123-456-7896', 'customer6@example.com', 'Address 6', '2023-06-28 00:00:00')
(7, 'Customer7', '123-456-7897', 'customer7@example.com', 'Address 7', '2022-04-23 00:00:00')
(8, 'Customer8', '123-456-7898', 'customer8@example.com', 'Address 8', '2022-01-20 00:00:00')
(9, 'Customer9', '123-456-7899', 'customer9@example.com', 'Address 9', '2022-11-30 00:00:00')
(10, 'Customer10', '123-456-78910', 'customer10@example.com', 'Address 10', '2023-12-15 00:00:00')

FactOrder (first 10 rows):
(1, 9, 8, 299,

In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('ecommerce.db')

# Function to execute and display SQL query results
def execute_query(query):
    df = pd.read_sql_query(query, conn)
    return df

# 1. Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months
query1 = """
SELECT
    c.CustomerName,
    AVG(o.TotalAmount) AS AverageOrderAmount
FROM
    FactOrder o
JOIN
    DimCustomer c ON o.CustomerID = c.CustomerID
JOIN
    DimDate d ON o.OrderDateID = d.DateID
WHERE
    d.Date >= DATE('2024-06-07', '-6 months')
GROUP BY
    c.CustomerName
ORDER BY
    AverageOrderAmount DESC
LIMIT 5;
"""
print("Top 5 customers with highest average order amounts in the last 6 months:")
display(execute_query(query1))

# 2. Retrieve the list of customers whose order value is lower this year as compared to the previous year
query2 = """
WITH OrderTotals AS (
    SELECT
        o.CustomerID,
        d.Year,
        SUM(o.TotalAmount) AS TotalAmount
    FROM
        FactOrder o
    JOIN
        DimDate d ON o.OrderDateID = d.DateID
    WHERE
        d.Year IN (2023, 2024)
    GROUP BY
        o.CustomerID, d.Year
)
SELECT
    c.CustomerName
FROM
    OrderTotals ot1
JOIN
    OrderTotals ot2 ON ot1.CustomerID = ot2.CustomerID AND ot1.Year = 2023 AND ot2.Year = 2024
JOIN
    DimCustomer c ON ot1.CustomerID = c.CustomerID
WHERE
    ot2.TotalAmount < ot1.TotalAmount;
"""
print("Customers whose order value is lower this year compared to the previous year:")
display(execute_query(query2))

# 3. Create a table showing cumulative purchases by a particular customer, broken down by product category
query3 = """
CREATE VIEW IF NOT EXISTS CustomerCumulativePurchases AS
SELECT
    c.CustomerID,
    c.CustomerName,
    p.Category,
    SUM(o.TotalAmount) AS CumulativePurchaseAmount
FROM
    FactOrder o
JOIN
    DimCustomer c ON o.CustomerID = c.CustomerID
JOIN
    DimProductVariant pv ON o.ProductVariantID = pv.ProductVariantID
JOIN
    DimProduct p ON pv.ProductID = p.ProductID
GROUP BY
    c.CustomerID, c.CustomerName, p.Category;
"""

# Execute the view creation query
conn.execute(query3)

# Query the view for a specific customer (e.g., CustomerID = 1)
query3_display = "SELECT * FROM CustomerCumulativePurchases WHERE CustomerID = 1;"
print("Cumulative purchases by a particular customer, broken down by product category:")
display(execute_query(query3_display))

# 4. Retrieve the list of top 5 selling products, further bifurcate the sales by product variants
query4 = """
WITH ProductSales AS (
    SELECT
        pv.ProductID,
        p.ProductName,
        pv.ProductVariantID,
        pv.VariantName,
        SUM(o.Quantity) AS TotalQuantitySold
    FROM
        FactOrder o
    JOIN
        DimProductVariant pv ON o.ProductVariantID = pv.ProductVariantID
    JOIN
        DimProduct p ON pv.ProductID = p.ProductID
    GROUP BY
        pv.ProductID, p.ProductName, pv.ProductVariantID, pv.VariantName
)
SELECT
    ps.ProductName,
    ps.VariantName,
    ps.TotalQuantitySold
FROM
    ProductSales ps
JOIN (
    SELECT
        ProductID,
        SUM(TotalQuantitySold) AS TotalProductQuantity
    FROM
        ProductSales
    GROUP BY
        ProductID
    ORDER BY
        TotalProductQuantity DESC
    LIMIT 5
) top5 ON ps.ProductID = top5.ProductID
ORDER BY
    top5.TotalProductQuantity DESC, ps.VariantName;
"""
print("Top 5 selling products, bifurcated by product variants:")
display(execute_query(query4))

# Close the connection
conn.close()


Top 5 customers with highest average order amounts in the last 6 months:


,CustomerName,AverageOrderAmount
0,Customer1,1013.438121
1,Customer3,965.772705
2,Customer9,905.683577
3,Customer7,903.303283
4,Customer5,877.951678


Customers whose order value is lower this year compared to the previous year:


,CustomerName


Cumulative purchases by a particular customer, broken down by product category:


,CustomerID,CustomerName,Category,CumulativePurchaseAmount
0,1,Customer1,Category1,22764.455605
1,1,Customer1,Category2,14072.565621
2,1,Customer1,Category3,12055.507648
3,1,Customer1,Category4,12510.383632


Top 5 selling products, bifurcated by product variants:


,ProductName,VariantName,TotalQuantitySold
0,ProductB,VariantB1,336
1,ProductB,VariantB2,359
2,ProductA,VariantA1,338
3,ProductA,VariantA2,271
4,ProductD,VariantD1,337
5,ProductH,VariantH1,334
6,ProductC,VariantC1,324
